In [0]:
spark.sql("USE CATALOG hp_az_catalog")
spark.sql("USE SCHEMA hp_iris_ml")

In [0]:
%pip install mlflow=='3.4.0'
dbutils.library.restartPython()

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import (
  ServedEntityInput,
  EndpointCoreConfigInput
)
from databricks.sdk.errors import ResourceDoesNotExist
from mlflow import MlflowClient


In [0]:
# Promote the model version to Champion
client = MlflowClient()
model_name = "hp_az_catalog.hp_iris_ml.irisclassifierdemo"
model_version = "5"
client.set_registered_model_alias(
    f'{model_name}',
    "Champion",
    model_version
)


In [0]:
# Create a serving endpoint for the model
# REQUIRED: Enter serving endpoint name
serving_endpoint_name = model_name.replace('.', '-') + "-endpoint"

w = WorkspaceClient()  # Assumes DATABRICKS_HOST and DATABRICKS_TOKEN are set
served_entities=[
  ServedEntityInput(
    entity_name=model_name,
    entity_version=model_version,
    workload_size="Small",
    scale_to_zero_enabled=True
  )
]

# Update serving endpoint if it already exists, otherwise create the serving endpoint
try:
  w.serving_endpoints.update_config(name=serving_endpoint_name, served_entities=served_entities)
except ResourceDoesNotExist:
  w.serving_endpoints.create(name=serving_endpoint_name, config=EndpointCoreConfigInput(served_entities=served_entities))


In [0]:
w.serving_endpoints